In [1]:
from geoband.API import *
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import geopandas as gpd
import json
from pandas import json_normalize
import folium
from folium.plugins import MarkerCluster
import matplotlib.pyplot as plt
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
from folium import Choropleth
import matplotlib
from datetime import datetime 
# GetCompasData로 주피터 노트북 경로에 파일 저장
from shapely.geometry import Point, Polygon, LineString
from fiona.crs import from_string
matplotlib.rcParams['font.family']="Malgun Gothic"

In [2]:
# 서울
서울=["강남구","강동구","강북구","강서구","관악구","광진구","구로구","금천구","노원구","도봉구"
,"동대문구","동작구","마포구","서대문구","서초구","성동구","성북구","송파구","양천구","영등포구"
,"용산구","은평구","중구","중랑구", "종로구"]

# 수원
수원=["장안구","팔달구","권선구","영통구"]

# 창원
창원=["의창구", "성산구", "마산회원구", "마산합포구", "진해구"]

# 진주
진주=["진주시"]

# 전체
전체=["강남구","강동구","강북구","강서구","관악구","광진구","구로구","금천구","노원구","도봉구", 
    "동대문구","동작구","마포구","서대문구","서초구","성동구","성북구","송파구","양천구","영등포구",
    "용산구","은평구","중구","중랑구", "종로구", "장안구","팔달구","권선구","영통구", "의창구", "성산구", 
    "마산회원구", "마산합포구", "진해구", "진주시"]

In [3]:
# 행정구역 경계

SIG = gpd.read_file("전국시군구_경계구역.json")
HJD = gpd.read_file("전국행정동_경계구역.geojson")

SIG_4 = SIG[(SIG["SIG_CD"].str[:2] == "11") | 
            (SIG["SIG_KOR_NM"].str.split(" ").str[0] == "수원시") | 
            (SIG["SIG_KOR_NM"].str.split(" ").str[0] == "창원시") |
            (SIG["SIG_KOR_NM"].str.split(" ").str[0] == "진주시")]
# 서울 수원 창원 진주의 시군구 경계구역만 가져오기

SIG_4["sgg_nm"] =  SIG_4["SIG_KOR_NM"].apply(lambda x : x.split(" ")[-1])
# '창원시 의창구 -> 의창구'

HJD_4 = HJD[(HJD["sido"] == "11") |
        (HJD["sggnm"] == "수원시장안구") | (HJD["sggnm"] == "수원시권선구") | (HJD["sggnm"] == "수원시팔달구") | 
        (HJD["sggnm"] == "수원시영통구") | (HJD["sggnm"] == "진주시") | (HJD["sggnm"] == "창원시의창구") | (HJD["sggnm"] == "창원시성산구") |
        (HJD["sggnm"] == "창원시마산합포구") | (HJD["sggnm"] == "창원시마산회원구")| (HJD["sggnm"] == "창원시진해구")]

# 서울 수원 창원 진주의 행정동 경계구역만 가져오기

관할경찰서-> 관할치안센터 

관할경찰청-> 관할경찰서 

In [4]:
others_police_box_area = gpd.read_file('12.4개시도_치안안전센터_관할경계.geojson')
others_police_box_area.columns=['관할치안센터','관할경찰서','geometry']
others_police_box_area=others_police_box_area.drop_duplicates(['관할치안센터'])
others_police_box_area.reset_index(drop=True,inplace=True)

def point_police_box(data_df,oters_police_box_area,col):
    df = data_df.copy()

    epsg4326 = from_string("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
    df['geometry'] = df.apply(lambda x : Point([x['lon'], x['lat']]), axis=1)
    df = gpd.GeoDataFrame(df, geometry='geometry', crs = epsg4326)
    
    police_box_result = gpd.sjoin(others_police_box_area, df, how='inner', op="intersects").drop(["index_right"],axis=1)
    
    police_box_result_groupby = police_box_result.groupby("관할치안센터").agg({col:"count"}).reset_index().rename(columns={col:"cnt"})
    
    return police_box_result,police_box_result_groupby

In [5]:
# 원래 데이터
suwon_building = pd.read_csv('26.수원시_건축물대장(표제부).csv')
others_park = pd.read_csv('27.4개시도_공원현황.csv')
others_one_person = pd.read_csv('33.4개시도_1인가구수.csv')
others_basic_recipient = pd.read_csv('34.4개시도_기초수급자현황.csv')
others_sex_age_pop = pd.read_csv('35.4개시도_성연령별_총인구분포.csv')
others_foreigner_pop = pd.read_csv('36.4개시도_인구데이터(외국인).csv')
others_suicide = pd.read_csv('37.4개시도_자살율.csv')
others_fire = pd.read_csv('38.4개시도_화재발생통계.csv')
others_police = pd.read_csv('39.4개시도_경찰관서현황.csv')
others_playground = pd.read_csv('40.4개시도_어린이놀이시설현황.csv')
others_land_use_plan = gpd.read_file('41.4개시도_토지이용계획도.geojson')
others_convenience = gpd.read_file('42.4개시도_편의점현황.geojson')
suwon_child_center = pd.read_csv('15.수원시_지역아동센터시설현황.csv')
suwon_youth_study = pd.read_csv('16.수원시_청소년공부방현황.csv')
suwon_library = pd.read_csv('17.수원시_도서관현황.csv')
suwon_free_cafeteria =pd.read_csv('18.수원시_무료급식소현황.csv')
suwon_child_free_cafeteria = pd.read_csv('19.수원시_아동복지급식소현황.csv')
suwon_commercial_area = pd.read_excel('25.수원시_상권(업종)현황.xlsx')
suwon_grid = gpd.read_file('32.수원시_표준격자(250M_x_250M).geojson')

# 전처리_나연.ipynb에서 추가한 데이터
others_park_sggnm=pd.read_csv('27.4개시도_공원현황_구별종류별.csv')
others_basic_recipient_age_range=pd.read_csv('34.4개시도_기초수급자현황_연령대.csv')
others_sex_age_pop_range=pd.read_csv('35.4개시도_성연령별_총인구분포_연령대.csv')
others_sex_age_pop_prod=pd.read_csv('35.4개시도_성연령별_총인구분포_생산연령.csv')
others_sex_age_pop_stat=pd.read_csv('35.4개시도_성연령별_총인구분포_통계값.csv')
others_police_sggnm=pd.read_csv('39.4개시도_경찰관서현황_구별.csv')
others_playground_sggnm=pd.read_csv('40.4개시도_어린이놀이시설현황_구별종류별.csv')
others_convenience_company=pd.read_csv("42.4개시도_편의점현황_업체별수.csv")

In [6]:
# 추가 데이터(이전 대회)
car_accident=pd.read_csv('11.5개시도_교통사고현황.csv')
police_num=pd.read_csv('12.경찰서별_지구대별_인원현황.csv')
safe_feeling=pd.read_csv('4.체감안전도_통계(2017~2019).csv')
crime_commit=pd.read_csv('6.범죄발생원표.csv')
crime_caught=pd.read_csv('7.범죄검거원표.csv')
crimial=pd.read_csv('8.범죄피의자원표.csv')
latest_school=pd.read_csv('19.5개시도_최종학력통계.csv')

In [7]:
police_box_4sido = others_police_box_area.copy()
police_box_4sido = police_box_4sido[['관할치안센터','관할경찰서']]
police_box_4sido.sort_values(by=["관할치안센터"], inplace=True)
police_box_4sido.reset_index(inplace = True,drop=True)

지역명 매핑

In [8]:
area_list = []
for i in range(len(police_box_4sido)):
    area_list.append(police_box_4sido['관할경찰서'][i][0:2])
for i in range(len(area_list)):
    if (area_list[i]=='마산') | (area_list[i] == '진해'):
        area_list[i] = '창원'
police_box_4sido['지역'] = area_list


In [10]:
others_park_result, others_park_groupby=point_police_box(others_park,others_police_box_area,'park_gbn')
others_park_result=others_park_result.groupby(['관할치안센터', 'park_gbn']).count()
others_park_result.reset_index(inplace=True)
others_park_result=others_park_result[['관할치안센터','park_gbn','park_nm']]
others_park_result

,관할치안센터,park_gbn,park_nm
0,가락지구대,근린공원,4
1,가락지구대,어린이공원,13
2,가산파출소,어린이공원,3
3,가양지구대,근린공원,10
4,가양지구대,어린이공원,18
...,...,...,...
657,회원파출소,어린이공원,6
658,휘경파출소,어린이공원,1
659,흑석지구대,근린공원,3
660,흑석지구대,소공원,5


###  치안센터의 공원 개수 feature 생성

In [11]:
police_box_4sido['전체공원']=0
for i in range(len(others_park_groupby)):
    for j in range(len(police_box_4sido)):
        if others_park_groupby.loc[i, '관할치안센터'] == police_box_4sido.loc[j, '관할치안센터']:
                police_box_4sido.loc[j, '전체공원']=others_park_groupby.loc[i, 'cnt']
police_box_4sido['어린이공원']=0
police_box_4sido['근린공원']=0
police_box_4sido['소공원']=0
police_box_4sido['문화공원']=0
police_box_4sido['역사공원']=0
police_box_4sido['체육공원']=0

In [12]:
for i in range(len(others_park_result)):
    for j in range(len(police_box_4sido)):
        if others_park_result.loc[i, '관할치안센터'] == police_box_4sido.loc[j, '관할치안센터'] and others_park_result.loc[i, 'park_gbn'] == '근린공원':
            police_box_4sido.loc[j, '근린공원']=others_park_result.loc[i, 'park_nm']
        elif others_park_result.loc[i, '관할치안센터'] == police_box_4sido.loc[j, '관할치안센터'] and others_park_result.loc[i, 'park_gbn'] == '소공원':
            police_box_4sido.loc[j, '소공원']=others_park_result.loc[i, 'park_nm']
        elif others_park_result.loc[i, '관할치안센터'] == police_box_4sido.loc[j, '관할치안센터'] and others_park_result.loc[i, 'park_gbn'] == '어린이공원':
            police_box_4sido.loc[j, '어린이공원']=others_park_result.loc[i, 'park_nm']
        elif others_park_result.loc[i, '관할치안센터'] == police_box_4sido.loc[j, '관할치안센터'] and others_park_result.loc[i, 'park_gbn'] == '문화공원':
            police_box_4sido.loc[j, '문화공원']=others_park_result.loc[i, 'park_nm']
        elif others_park_result.loc[i, '관할치안센터'] == police_box_4sido.loc[j, '관할치안센터'] and others_park_result.loc[i, 'park_gbn'] == '역사공원':
            police_box_4sido.loc[j, '역사공원']=others_park_result.loc[i, 'park_nm']
        elif others_park_result.loc[i, '관할치안센터'] == police_box_4sido.loc[j, '관할치안센터'] and others_park_result.loc[i, 'park_gbn'] == '체육공원':
            police_box_4sido.loc[j, '체육공원']=others_park_result.loc[i, 'park_nm']


### 40.4개시도_어린이놀이시설현황/40.4개시도_어린이놀이시설현황_구별종류별

In [13]:
others_playground_result, others_playground_groupby=point_police_box(others_playground,others_police_box_area,'fac_loc')
# others_playground_result_=others_playground_result.groupby(['관할치안센터', 'fac_loc']).count()
# others_playground_result_.reset_index(inplace=True)
# others_playground_result_=others_playground_result_[['관할치안센터','fac_loc', 'fac_nm']]
# others_playground_result_.rename(columns={'fac_nm':'count'}, inplace=True)
# others_playground_result_

,관할치안센터,fac_loc,count
0,가락지구대,놀이제공영업소,1
1,가락지구대,도시공원,15
2,가락지구대,식품접객업소,1
3,가락지구대,어린이집,4
4,가락지구대,유치원,2
...,...,...,...
1773,흑석지구대,어린이집,2
1774,흑석지구대,유치원,5
1775,흑석지구대,종교시설,1
1776,흑석지구대,주택단지,39


In [18]:
police_box_4sido['놀이시설']=0
for i in range(len(others_playground_groupby)):
    for j in range(len(police_box_4sido)):
        if others_playground_groupby.loc[i, '관할치안센터'] == police_box_4sido.loc[j, '관할치안센터']:
                police_box_4sido.loc[j, '놀이시설']=others_playground_groupby.loc[i, 'cnt']

### 42.4개시도_편의점현황/42.4개시도_편의점현황_업체별수

In [14]:
others_convenience_result, others_convenience_groupby=point_police_box(others_convenience,others_police_box_area,'KSIC_NM')
others_convenience_result=others_convenience_result.groupby(['관할치안센터', 'KSIC_NM']).count()
others_convenience_result.reset_index(inplace=True)
others_convenience_result=others_convenience_result[['관할치안센터','KSIC_NM','BUSINESS_NM']]
others_convenience_result

,관할치안센터,KSIC_NM,BUSINESS_NM
0,가락지구대,체인화 편의점,52
1,가리봉파출소,체인화 편의점,9
2,가산파출소,체인화 편의점,95
3,가양지구대,기타 대형 종합 소매업,1
4,가양지구대,체인화 편의점,105
...,...,...,...
323,회원파출소,체인화 편의점,19
324,회현파출소,체인화 편의점,9
325,효제파출소,체인화 편의점,9
326,휘경파출소,체인화 편의점,35


In [15]:
police_box_4sido['편의점']=0
for i in range(len(others_convenience_groupby)):
    for j in range(len(police_box_4sido)):
        if others_convenience_groupby.loc[i, '관할치안센터'] == police_box_4sido.loc[j, '관할치안센터']:
                police_box_4sido.loc[j, '편의점']=others_convenience_groupby.loc[i, 'cnt']